In [1]:
from agent.llm import LLMClient
from agent.prompt import prompt1,prompt2 

from schema.response import parse_llm_json,user_xml,log_xml,context_to_text,log_data_xml

from state.build_user_profile import build_user_profile
from state.select_logs import select_relevant_logs
from state.build_file_index import build_file_index

from derived.build_derived_logs import build_derived_logs

import json
from datetime import date
import threading
def trigger_user_profile_async():
    """
    后台异步触发用户画像更新：
    - 不阻塞
    - 可被多次安全调用
    """
    t = threading.Thread(
        target=build_user_profile,
        daemon=True
    )
    t.start()
def get_log(user_input: str) -> str:
    log_text = select_relevant_logs(user_input)
    if len(log_text) == 0:
        return ''
    return log_data_xml(log_text)


In [3]:
llm = LLMClient()
trigger_user_profile_async()
#三个可变的全局变量
current_state = 'S1'#自由派 S2进入日志确定阶段
context = []#之前的聊天记录
draft_log = None#日志草稿，待用户确认
#当前日期
today = date.today()#今日日期
cur_date = f'<current_date>{today}</current_date>'
#对话流
while True:
    user_input = input().strip()#获取用户输入
    if user_input=='stop':
        break
    if not user_input:
        continue 
    context.append({'role':'user','content':user_input})#context
    if user_input.startswith('-'):
        log_previous = get_log(user_input)#获取相关日志数据
    else:
        log_previous = ''
    
    if current_state=='S1':##自由派
        #判断是日志还是正常聊天的节点
        if len(context) == 1:
            previous_context = str(context[0])
        else:
            previous_context = context_to_text(context)
        result = parse_llm_json(llm.generate(log_previous+previous_context+prompt1+cur_date+user_xml(user_input)))
        if result['type']=='1-1':#1-1日志写入，1-2正常聊天
            #调整状态和之前对话
            context.pop()
            current_state='S2'
            #获取日志草稿，供下一轮节点2问询
            draft_log = result['content']
            print(result['content'])
        elif result['type'] == '1-2':
            context.append({'role':'assistant','content':result['content']})
            print(result['content'])
        else:
            print('不好意思，大模型运行失败')
            print('Sorry, our LLM has went wrong')
        
    else:#节点2，确定日志内容
        result = llm.generate(
            log_previous+log_xml(draft_log)+prompt2+user_xml(user_input)
        )
        # print(result)
        result = parse_llm_json(result)
        if result['type']=='2-1':#保存日志
            path = './storage/'+result['content'][0]
            text = result['content'][1]
            with open(path,'w',encoding='utf-8') as f:
                f.write(text)
            print('好的，该日志已保存!')
            print('Okay, the log has been saved!')
            #更新状态和变量
            context.pop()#节点2不是正常聊天的上下文
            current_state = 'S1'#确认完，日志节点2循环完成
            draft_log = None#清空草稿
            build_file_index()#更新日志文件index
            build_derived_logs()#更新日志合并文件，便于展示原始日志内容
        elif result['type']=='2-2':#用户提出修改意见，更新日志草稿，再次进入循环
            context.pop()#修改日志不在正常聊天范围内
            draft_log = result['content']#更新日志草稿
            print(result['content'])
        elif result['type']=='2-3':#用户正常聊天，跳出循环
            context.append({'role':'assistant','content':result['content']})
            current_state='S1'#转向聊天的自由派
            










我判断这是一条【事件】日志。

类型：事件
日期：2026-01-04
事件日期：2026-01-09
内容：准备好去美国的东西。

请确认以上信息是否正确，或者您需要修改什么吗？
好的，该日志已保存!
Okay, the log has been saved!
